# Analyse results of custom blank model

## 1: Setup 

### 1.1: Load models and packages

In [19]:
import os
import pandas as pd
from pathlib import Path

### 1.2: Set up test and results filepaths

In [39]:
# Load the results of this model
returned_filepath = "../../../data/hand_annotated_abstracts/results/"
expected_filename = "../../../data/hand_annotated_abstracts/answers/hand_annotated_answers.csv"


## 2: Analysis of best custom model

### 2.1: Load best custom model results

In [31]:
custom_en_blank_results_df = pd.read_csv(os.path.join(returned_filepath, "custom_en_blank.csv"))
custom_en_blank_results_df.head(5)

,corpus,test_ent_text,test_ent_label,test_ent_start,test_ent_end,id,pmid
0,Trace Element Uptake and Accumulation in the M...,Hypericum perforatum L. Across Different Geoli...,scientific,60,125,36,30043286
1,Trace Element Uptake and Accumulation in the M...,St. John's wort,common,781,796,36,30043286
2,Improvements in estrogen deficiency-induced hy...,gut microbiota,common,120,134,37,33383372
3,Improvements in estrogen deficiency-induced hy...,HP extract,common,332,342,37,33383372
4,Improvements in estrogen deficiency-induced hy...,LDL-C,common,1844,1839,37,33383372


In [32]:
# Split of entity labels
custom_en_blank_results_df.value_counts("test_ent_label")

test_ent_label
scientific    27
common        18
dtype: int64

In [33]:
# Total number of entities returned
len(custom_en_blank_results_df)

45

In [34]:
# Number of entities and unique entities for each abstract
custom_en_blank_results_df[["id", "test_ent_label"]].groupby("id").agg(["count", "nunique"])

test_ent_label        
            count nunique
id                       
23              4       1
24              2       2
25              7       2
26              4       2
27              2       2
28              2       2
29              2       2
31              2       2
32              4       2
33              5       1
34              3       2
35              3       1
36              2       2
37              3       1

In [37]:
# Rename columns
list(custom_en_blank_results_df.columns)
columns = [
    "id",
    "test_ent_text",
    "test_ent_label",
    "test_ent_start",
    "test_ent_end"
]

custom_en_blank_results_df = custom_en_blank_results_df[columns]

columns = {
    "id": "id",
    "test_ent_text": "ent_text",
    "test_ent_label": "ent_label",
    "test_ent_start": "ent_start",
    "test_ent_end": "ent_end",
}

custom_en_blank_results_df = custom_en_blank_results_df.rename(columns=columns)

### 2.2 Load expected file and join to test results

In [62]:
expected_df = pd.read_csv(expected_filename)
# Rename columns
list(expected_df.columns)
columns = [
    "id",
    "hand_ent_text",
    "hand_ent_label",
    "hand_ent_start",
    "hand_ent_end"
]

expected_df = expected_df[columns]
columns = {
    "id": "id",
    "hand_ent_text": "ent_text",
    "hand_ent_label": "ent_label",
    "hand_ent_start": "ent_start",
    "hand_ent_end": "ent_end",
}

expected_df = expected_df.rename(columns=columns)

expected_df.head(5)

,id,ent_text,ent_label,ent_start,ent_end
0,23,Centella Asiatica (L.) Urb.,scientific,23,50
1,23,Centella asiatica,common,129,146
2,24,Centella asiatica (L.) Urban,scientific,168,196
3,25,Centella asiatica,common,40,57
4,25,Centella asiatica (L.) Urb.,scientific,110,137


In [63]:
custom_en_blank_and_expected_df = custom_en_blank_results_df.merge(expected_df, on=["id", "ent_text", "ent_start", "ent_end"], how="outer", suffixes=("_custom_en", "_hand"))

# Tidy column headings and rearrange the columns for reading
custom_en_blank_and_expected_df = custom_en_blank_and_expected_df.rename(columns={"ent_label": "ent_label_custom_en"})
# list(results_df.columns)

columns = [
    "id",
    "ent_text",
    "ent_label_custom_en",
    "ent_label_hand",
    "ent_start",
    "ent_end"
]
custom_en_blank_and_expected_df = custom_en_blank_and_expected_df[columns]
custom_en_blank_and_expected_df.head(30)

,id,ent_text,ent_label_custom_en,ent_label_hand,ent_start,ent_end
0,36,Hypericum perforatum L. Across Different Geoli...,scientific,NaN,60,125
1,36,St. John's wort,common,common,781,796
2,37,gut microbiota,common,NaN,120,134
3,37,HP extract,common,NaN,332,342
4,37,LDL-C,common,NaN,1844,1839
5,27,Protective effects,scientific,NaN,0,18
6,27,silymarin,common,NaN,802,811
7,29,Hypericum perforatum L.,scientific,scientific,0,23
8,29,St. John's,common,NaN,180,190
9,23,Centella Asiatica (L.) Urb.,scientific,scientific,23,50


### 2.3 Analysis of all results

In [69]:
# Common calculations
def calculate_precision(tp: int, fp: int) -> float:
    return tp / (tp + fp + 1e-100)

def calculate_recall(tp: int, fn: int) -> float:
    return tp / (tp + fn + 1e-100)

def calculate_f1(precision: float, recall: float) -> float:
    return (2 * precision * recall) / (precision + recall + 1e-100)

Get TP, FP, and FN for custom model

In [44]:
# Get TP, FP, FN for en_core_sci_md_results_and_expected_df
true_postive = (custom_en_blank_and_expected_df["ent_label_custom_en"].notnull() & custom_en_blank_and_expected_df["ent_label_hand"].notnull())
false_postive = (custom_en_blank_and_expected_df["ent_label_custom_en"].notnull() & custom_en_blank_and_expected_df["ent_label_hand"].isnull())
false_negative = (custom_en_blank_and_expected_df["ent_label_custom_en"].isnull() & custom_en_blank_and_expected_df["ent_label_hand"].notnull())

In [48]:
custom_en_blank_and_expected_df[true_postive]
custom_en_blank_results_tp = len(custom_en_blank_and_expected_df[true_postive])
custom_en_blank_results_tp

12

In [49]:
custom_en_blank_and_expected_df[false_postive]
custom_en_blank_results_fp = len(custom_en_blank_and_expected_df[false_postive])
custom_en_blank_results_fp

33

In [50]:
custom_en_blank_and_expected_df[false_negative]
custom_en_blank_results_fn = len(custom_en_blank_and_expected_df[false_negative])
custom_en_blank_results_fn

42

In [70]:
print(len(custom_en_blank_results_df), custom_en_blank_results_tp, custom_en_blank_results_fp, custom_en_blank_results_fn)

45 12 33 42


In [71]:
calculate_precision(tp=custom_en_blank_results_tp, fp=custom_en_blank_results_fp)

0.26666666666666666

In [72]:
calculate_recall(tp=custom_en_blank_results_tp, fn=custom_en_blank_results_fn)

0.2222222222222222

In [73]:
calculate_f1(precision=calculate_precision(tp=custom_en_blank_results_tp, fp=custom_en_blank_results_fp), recall=calculate_recall(tp=custom_en_blank_results_tp, fn=custom_en_blank_results_fn))

0.2424242424242424

### 2.4 Further analysis

In [56]:
# Get hits and misses with types of entities
custom_en_blank_results_tp_common = (custom_en_blank_and_expected_df["ent_label_custom_en"].notnull() & (custom_en_blank_and_expected_df["ent_label_hand"]=="common"))
custom_en_blank_results_tp_scientific = (custom_en_blank_and_expected_df["ent_label_custom_en"].notnull() & (custom_en_blank_and_expected_df["ent_label_hand"]=="scientific"))

custom_en_blank_results_fn_common = (custom_en_blank_and_expected_df["ent_label_custom_en"].isnull() & (custom_en_blank_and_expected_df["ent_label_hand"]=="common"))
custom_en_blank_results_fn_scientific = (custom_en_blank_and_expected_df["ent_label_custom_en"].isnull() & (custom_en_blank_and_expected_df["ent_label_hand"]=="scientific"))


In [57]:
custom_en_blank_and_expected_df[custom_en_blank_results_tp_common]
len(custom_en_blank_and_expected_df[custom_en_blank_results_tp_common])

2

In [58]:
custom_en_blank_and_expected_df[custom_en_blank_results_tp_scientific]
len(custom_en_blank_and_expected_df[custom_en_blank_results_tp_scientific])

10

In [59]:
custom_en_blank_and_expected_df[custom_en_blank_results_fn_common]
len(custom_en_blank_and_expected_df[custom_en_blank_results_fn_common])

28

In [60]:
custom_en_blank_and_expected_df[custom_en_blank_results_fn_scientific]
len(custom_en_blank_and_expected_df[custom_en_blank_results_fn_scientific])

14